# LangChain and LLM dolly

In [ ]:
!pip install --upgrade langchain

In [ ]:
%pip install "transformers[torch]"

In [ ]:
import torch
from transformers import pipeline


# databricks/dolly-v2-3b
# databricks/dolly-v2-7b
# databricks/dolly-v2-2-8b
# databricks/dolly-v2-12b

# OpenAssistant/falcon-7b-sft-mix-2000
# OpenAssistant/falcon-40b-sft-mix-1226


generate_text = pipeline(model="databricks/dolly-v2-3b",
                         torch_dtype=torch.bfloat16,
                         trust_remote_code=True,
                         device_map="auto",
                         return_full_text=True)

In [ ]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline

# Installing Whisper for speaking-to-text trasncript

In [ ]:
# Install the latest commit from Github whisper repository
! pip install git+https://github.com/openai/whisper.git -q

In [ ]:
# Installing command-line tool ffmpeg
!sudo apt update && sudo apt install ffmpeg

# Install the Web UI Toolkit

In [ ]:
! pip install gradio -q

# The App

In [ ]:
import whisper
import gradio as gr
import time

In [ ]:
hf_pipeline = HuggingFacePipeline(pipeline=generate_text)

English_review_template = """You are a professional examiner who assesses the English language proficiency of candidates taking an English test.
You asked this question: {question} \
The answer was: {text}.

Evaluate the answer based on the following criteria:\

- Coherence: Assess how well the candidate's answer is connected to the question.
- Word Choice: Assess if the candidate uses appropriate words and phrases to communicate.
- Vocabulary Range: Assess if the candidate uses a variety of vocabulary.
- Sentence Structure: Assess the candidate's ability to use a variety of sentence structures appropriately.
- Grammatical Accuracy: Explain how well did the candidate formed grammatically correct sentences?
- Grammatical Errors: List and explain each grammar error in the candidate's answer.
"""

prompt_template = PromptTemplate.from_template(English_review_template)

llm_chain = LLMChain(llm=hf_pipeline, prompt=prompt_template)

In [ ]:
def English_feedback(Question, audio):
  model = whisper.load_model("base")
  text = model.transcribe(audio)
  response = llm_chain.run({'text': text, 'question': Question})
  return text["text"], response

In [ ]:
demo = gr.Interface(
    title = 'English Speaking AI-examiner ',
    fn=English_feedback,
    inputs=["text", gr.inputs.Audio(source="microphone", type="filepath")],
    outputs=[ "textbox", "textbox"])

demo.launch(share=True)